In [3]:
# MIT License
# 
# Copyright (c) 2016 Michaël Defferrard
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Baselines Notebook from FMA

In [4]:
import time
import os

import IPython.display as ipd
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import keras
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier

import fma_utils as utils

In [5]:
AUDIO_DIR = '../data/fma_medium'

tracks = utils.load('../data/fma_metadata/tracks.csv')
features = utils.load('../data/fma_metadata/features.csv')
echonest = utils.load('../data/fma_metadata/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, features.shape, echonest.shape

((106574, 52), (106574, 518), (13129, 249))

## Subset

In [6]:
subset = tracks.index[tracks['set', 'subset'] <= 'medium']

assert subset.isin(tracks.index).all()
assert subset.isin(features.index).all()

features_all = features.join(echonest, how='inner').sort_index(axis=1)
print('Not enough Echonest features: {}'.format(features_all.shape))

tracks = tracks.loc[subset]
features_all = features.loc[subset]

tracks.shape, features_all.shape

Not enough Echonest features: (13129, 767)


((25000, 52), (25000, 518))

In [7]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))

19922 training examples, 2505 validation examples, 2573 testing examples
Top genres (16): ['Blues', 'Classical', 'Country', 'Easy Listening', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Old-Time / Historic', 'Pop', 'Rock', 'Soul-RnB', 'Spoken']
All genres (151): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 30, 31, 32, 33, 36, 37, 38, 41, 42, 43, 45, 46, 47, 49, 53, 58, 63, 64, 65, 66, 70, 71, 74, 76, 77, 79, 81, 83, 85, 86, 88, 89, 90, 92, 94, 97, 98, 100, 101, 102, 103, 107, 109, 111, 113, 117, 118, 125, 130, 137, 138, 166, 167, 169, 171, 172, 174, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 214, 224, 232, 236, 240, 247, 250, 267, 286, 296, 297, 311, 314, 322, 337, 359, 360, 361, 362, 374, 378, 400, 401, 404, 428, 439, 440, 441, 442, 443, 456, 468, 491, 495, 502, 504, 514, 524, 538, 539, 542, 580, 602, 619, 651, 659, 695, 741, 763, 808, 810, 811, 906, 1032, 1060, 1193, 1235]


## Multiple Classifiers and feature sets

In [18]:
def pre_process(tracks, features, columns, multi_label=False, verbose=False):
    if not multi_label:
        # Assign an integer value to each genre.
        enc = LabelEncoder()
        labels = tracks['track', 'genre_top']
        #y = enc.fit_transform(tracks['track', 'genre_top'])
    else:
        # Create an indicator matrix.
        enc = MultiLabelBinarizer()
        labels = tracks['track', 'genres_all']
        #labels = tracks['track', 'genres']

    # Split in training, validation and testing sets.
    y_train = enc.fit_transform(labels[train])
    y_val = enc.transform(labels[val])
    y_test = enc.transform(labels[test])
    display(features)
    X_train = features.loc[train, columns].to_numpy()
    X_val = features.loc[val, columns].to_numpy()
    X_test = features.loc[test, columns].to_numpy()
    
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    
    # Standardize features by removing the mean and scaling to unit variance.
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_val)
    scaler.transform(X_test)
    
    return y_train, y_val, y_test, X_train, X_val, X_test

In [9]:
def test_classifiers_features(classifiers, feature_sets, multi_label=False):
    columns = list(classifiers.keys()).insert(0, 'dim')
    scores = pd.DataFrame(columns=columns, index=feature_sets.keys())
    times = pd.DataFrame(columns=classifiers.keys(), index=feature_sets.keys())
    for fset_name, fset in tqdm_notebook(feature_sets.items(), desc='features'):
        y_train, y_val, y_test, X_train, X_val, X_test = pre_process(tracks, features_all, fset, multi_label)
        scores.loc[fset_name, 'dim'] = X_train.shape[1]
        for clf_name, clf in classifiers.items():  # tqdm_notebook(classifiers.items(), desc='classifiers', leave=False):
            t = time.process_time()
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            scores.loc[fset_name, clf_name] = score
            times.loc[fset_name, clf_name] = time.process_time() - t
    return scores, times

def format_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: yellow' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.2%}', subset=pd.IndexSlice[:, scores.columns[1]:])

In [8]:
classifiers = {
    'LR': LogisticRegression(max_iter=1000),
    'kNN': KNeighborsClassifier(n_neighbors=200),
    'SVCrbf': SVC(kernel='rbf'),
    'SVCpoly1': SVC(kernel='poly', degree=1),
    'linSVC1': SVC(kernel="linear"),
    'linSVC2': LinearSVC(),
    #GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    'DT': DecisionTreeClassifier(max_depth=5),
    'RF': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    'AdaBoost': AdaBoostClassifier(n_estimators=10),
    'MLP1': MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000),
    'MLP2': MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=2000),
    'NB': GaussianNB(),
    'QDA': QuadraticDiscriminantAnalysis(),
}

feature_sets = {
#    'echonest_audio': ('echonest', 'audio_features'),
#    'echonest_social': ('echonest', 'social_features'),
#    'echonest_temporal': ('echonest', 'temporal_features'),
#    'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
#    'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
for name in features.columns.levels[0]:
    feature_sets[name] = name
feature_sets.update({
    'mfcc/contrast': ['mfcc', 'spectral_contrast'],
    'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
    'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
    'all_non-echonest': list(features.columns.levels[0])
})

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

c:\users\david\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, incr

c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\users\david\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\david\a

,dim,LR,kNN,SVCrbf,SVCpoly1,linSVC1,linSVC2,DT,RF,AdaBoost,MLP1,MLP2,NB,QDA
chroma_cens,84.000000,39.49%,37.50%,42.29%,38.63%,39.29%,39.02%,35.68%,33.81%,30.86%,37.97%,32.92%,9.99%,24.64%
chroma_cqt,84.000000,40.23%,40.03%,44.27%,39.95%,41.39%,40.46%,35.45%,36.88%,35.72%,43.65%,36.03%,1.55%,3.15%
chroma_stft,84.000000,43.72%,43.92%,48.31%,43.65%,44.35%,42.83%,39.88%,38.71%,35.25%,48.31%,39.10%,4.20%,5.67%
mfcc,140.000000,57.99%,54.99%,60.98%,59.66%,59.19%,56.63%,45.82%,43.96%,41.31%,50.25%,52.82%,41.86%,48.39%
rmse,7.000000,36.92%,38.52%,38.90%,37.70%,37.54%,37.35%,38.63%,38.36%,34.67%,38.87%,38.36%,11.78%,15.04%
spectral_bandwidth,7.000000,40.58%,45.39%,44.46%,40.38%,40.42%,40.61%,42.91%,43.10%,37.47%,45.39%,43.68%,36.18%,34.16%
spectral_centroid,7.000000,42.75%,45.36%,45.71%,42.09%,42.09%,42.21%,42.67%,44.42%,42.60%,47.26%,45.94%,33.31%,36.11%
spectral_contrast,49.000000,51.77%,49.55%,54.45%,49.59%,51.81%,49.09%,43.53%,43.72%,39.53%,52.08%,47.14%,39.41%,41.78%
spectral_rolloff,7.000000,41.78%,46.25%,47.53%,41.43%,41.62%,41.47%,45.36%,46.40%,41.66%,48.31%,46.87%,28.49%,28.53%
tonnetz,42.000000,40.11%,37.31%,42.25%,40.23%,40.15%,39.56%,35.91%,35.83%,34.16%,39.99%,32.45%,22.31%,23.05%


,LR,kNN,SVCrbf,SVCpoly1,linSVC1,linSVC2,DT,RF,AdaBoost,MLP1,MLP2,NB,QDA
chroma_cens,31.8906,10.2969,73.2812,58.3906,200.7812,68.7812,0.8750,0.1250,2.1562,134.8750,204.0469,0.0938,0.7500
chroma_cqt,30.4688,9.9375,66.5469,59.0000,267.7656,74.5469,0.8125,0.1094,2.0312,94.8906,263.1094,0.0781,0.7969
chroma_stft,35.3125,9.2812,61.6250,60.2500,182.1719,68.1562,0.8125,0.1094,1.9844,102.6562,270.6719,0.0781,0.7656
mfcc,55.6875,16.1562,74.1562,60.1250,177.1562,67.2812,1.6406,0.1250,3.8906,183.1094,101.4062,0.1250,1.7031
rmse,5.2344,0.5000,19.7500,10.2656,13.6250,11.1094,0.0781,0.1094,0.2969,54.1875,243.4531,0.0156,0.0000
spectral_bandwidth,4.9688,0.4375,19.4219,10.8594,16.3750,11.4844,0.0781,0.1094,0.2969,88.2656,154.5156,0.0156,0.0156
spectral_centroid,4.4375,0.4062,17.3281,11.6406,18.6406,11.3125,0.0781,0.1094,0.2969,73.5625,194.4219,0.0156,0.0000
spectral_contrast,17.2812,6.0625,31.5938,26.3594,62.7031,42.5156,0.5625,0.1094,1.4375,84.4219,230.2969,0.0312,0.2500
spectral_rolloff,4.3438,0.4219,18.2344,11.3281,16.6875,11.1719,0.0469,0.0938,0.2656,56.1562,223.9375,0.0156,0.0000
tonnetz,12.0781,5.0312,39.2812,28.3438,67.5469,45.4531,0.4844,0.1094,1.2344,79.5625,251.4531,0.0156,0.3281


In [9]:
scores.to_csv('./baseline_scores.csv')
times.to_csv('./baseline_times.csv')

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

classifiers = {
    'GradientBoost': GradientBoostingClassifier(max_depth=5, n_estimators=10),
    'XGB': XGBClassifier(max_depth=5, n_estimators=10)
}

feature_sets = {
#    'echonest_audio': ('echonest', 'audio_features'),
#    'echonest_social': ('echonest', 'social_features'),
#    'echonest_temporal': ('echonest', 'temporal_features'),
#    'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
#    'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
for name in features.columns.levels[0]:
    feature_sets[name] = name
feature_sets.update({
    'mfcc/contrast': ['mfcc', 'spectral_contrast'],
    'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
    'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
    'all_non-echonest': list(features.columns.levels[0])
})

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

c:\users\david\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


,dim,GradientBoost,XGB
chroma_cens,84.000000,36.03%,38.71%
chroma_cqt,84.000000,40.11%,40.23%
chroma_stft,84.000000,43.02%,44.89%
mfcc,140.000000,52.55%,55.77%
rmse,7.000000,39.49%,41.04%
spectral_bandwidth,7.000000,42.79%,44.93%
spectral_centroid,7.000000,45.32%,46.95%
spectral_contrast,49.000000,48.97%,51.54%
spectral_rolloff,7.000000,44.93%,47.38%
tonnetz,42.000000,40.30%,42.05%


,GradientBoost,XGB
chroma_cens,119.8281,71.2344
chroma_cqt,110.1094,62.0156
chroma_stft,103.9688,46.2188
mfcc,201.9844,101.2031
rmse,10.5938,17.0469
spectral_bandwidth,10.6250,17.4688
spectral_centroid,10.6406,17.4531
spectral_contrast,70.8125,39.8125
spectral_rolloff,8.9219,16.7656
tonnetz,61.8438,43.9531


In [13]:
from catboost import CatBoostClassifier

classifiers = {
    'CatBoost': CatBoostClassifier()
}

feature_sets = {
#    'echonest_audio': ('echonest', 'audio_features'),
#    'echonest_social': ('echonest', 'social_features'),
#    'echonest_temporal': ('echonest', 'temporal_features'),
#    'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
#    'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
for name in features.columns.levels[0]:
    feature_sets[name] = name
feature_sets.update({
    'mfcc/contrast': ['mfcc', 'spectral_contrast'],
    'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
    'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
    'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
    'all_non-echonest': list(features.columns.levels[0])
})

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

c:\users\david\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Learning rate set to 0.09217
0:	learn: 2.5985319	total: 394ms	remaining: 6m 33s
1:	learn: 2.4867479	total: 746ms	remaining: 6m 12s
2:	learn: 2.3980826	total: 1.1s	remaining: 6m 6s
3:	learn: 2.3323415	total: 1.46s	remaining: 6m 4s
4:	learn: 2.2711737	total: 1.8s	remaining: 5m 59s
5:	learn: 2.2219821	total: 2.13s	remaining: 5m 53s
6:	learn: 2.1793892	total: 2.46s	remaining: 5m 48s
7:	learn: 2.1449506	total: 2.8s	remaining: 5m 46s
8:	learn: 2.1148857	total: 3.13s	remaining: 5m 44s
9:	learn: 2.0872333	total: 3.46s	remaining: 5m 42s
10:	learn: 2.0644471	total: 3.78s	remaining: 5m 39s
11:	learn: 2.0433569	total: 4.11s	remaining: 5m 38s
12:	learn: 2.0247157	total: 4.47s	remaining: 5m 39s
13:	learn: 2.0070349	total: 4.8s	remaining: 5m 38s
14:	learn: 1.9921771	total: 5.14s	remaining: 5m 37s
15:	learn: 1.9777987	total: 5.46s	remaining: 5m 35s
16:	learn: 1.9656722	total: 5.81s	remaining: 5m 35s
17:	learn: 1.9522100	total: 6.15s	remaining: 5m 35s
18:	learn: 1.9397051	total: 6.49s	remaining: 5m 35s

158:	learn: 1.5284823	total: 56.2s	remaining: 4m 57s
159:	learn: 1.5267333	total: 56.5s	remaining: 4m 56s
160:	learn: 1.5248784	total: 56.9s	remaining: 4m 56s
161:	learn: 1.5224490	total: 57.2s	remaining: 4m 56s
162:	learn: 1.5204711	total: 57.6s	remaining: 4m 55s
163:	learn: 1.5192276	total: 57.9s	remaining: 4m 55s
164:	learn: 1.5176892	total: 58.3s	remaining: 4m 54s
165:	learn: 1.5153885	total: 58.7s	remaining: 4m 54s
166:	learn: 1.5138321	total: 59s	remaining: 4m 54s
167:	learn: 1.5128958	total: 59.3s	remaining: 4m 53s
168:	learn: 1.5109509	total: 59.6s	remaining: 4m 53s
169:	learn: 1.5088165	total: 60s	remaining: 4m 52s
170:	learn: 1.5073046	total: 1m	remaining: 4m 52s
171:	learn: 1.5050823	total: 1m	remaining: 4m 52s
172:	learn: 1.5029310	total: 1m 1s	remaining: 4m 51s
173:	learn: 1.5018488	total: 1m 1s	remaining: 4m 51s
174:	learn: 1.4995886	total: 1m 1s	remaining: 4m 50s
175:	learn: 1.4976295	total: 1m 2s	remaining: 4m 50s
176:	learn: 1.4955688	total: 1m 2s	remaining: 4m 50s
177

311:	learn: 1.2964443	total: 1m 52s	remaining: 4m 7s
312:	learn: 1.2953762	total: 1m 52s	remaining: 4m 7s
313:	learn: 1.2940812	total: 1m 53s	remaining: 4m 6s
314:	learn: 1.2926055	total: 1m 53s	remaining: 4m 6s
315:	learn: 1.2912054	total: 1m 53s	remaining: 4m 6s
316:	learn: 1.2898916	total: 1m 54s	remaining: 4m 5s
317:	learn: 1.2884081	total: 1m 54s	remaining: 4m 5s
318:	learn: 1.2862381	total: 1m 54s	remaining: 4m 5s
319:	learn: 1.2844792	total: 1m 55s	remaining: 4m 4s
320:	learn: 1.2827593	total: 1m 55s	remaining: 4m 4s
321:	learn: 1.2812381	total: 1m 55s	remaining: 4m 4s
322:	learn: 1.2799383	total: 1m 56s	remaining: 4m 3s
323:	learn: 1.2787506	total: 1m 56s	remaining: 4m 3s
324:	learn: 1.2776052	total: 1m 56s	remaining: 4m 2s
325:	learn: 1.2763649	total: 1m 57s	remaining: 4m 2s
326:	learn: 1.2749862	total: 1m 57s	remaining: 4m 2s
327:	learn: 1.2739824	total: 1m 58s	remaining: 4m 1s
328:	learn: 1.2723094	total: 1m 58s	remaining: 4m 1s
329:	learn: 1.2711640	total: 1m 58s	remaining:

464:	learn: 1.1253599	total: 2m 46s	remaining: 3m 11s
465:	learn: 1.1244851	total: 2m 46s	remaining: 3m 11s
466:	learn: 1.1234436	total: 2m 47s	remaining: 3m 10s
467:	learn: 1.1224043	total: 2m 47s	remaining: 3m 10s
468:	learn: 1.1207125	total: 2m 47s	remaining: 3m 10s
469:	learn: 1.1195398	total: 2m 48s	remaining: 3m 9s
470:	learn: 1.1189332	total: 2m 48s	remaining: 3m 9s
471:	learn: 1.1177662	total: 2m 48s	remaining: 3m 8s
472:	learn: 1.1160617	total: 2m 49s	remaining: 3m 8s
473:	learn: 1.1148436	total: 2m 49s	remaining: 3m 8s
474:	learn: 1.1138674	total: 2m 49s	remaining: 3m 7s
475:	learn: 1.1130842	total: 2m 50s	remaining: 3m 7s
476:	learn: 1.1123173	total: 2m 50s	remaining: 3m 7s
477:	learn: 1.1114745	total: 2m 50s	remaining: 3m 6s
478:	learn: 1.1102146	total: 2m 51s	remaining: 3m 6s
479:	learn: 1.1092116	total: 2m 51s	remaining: 3m 5s
480:	learn: 1.1077195	total: 2m 52s	remaining: 3m 5s
481:	learn: 1.1061594	total: 2m 52s	remaining: 3m 5s
482:	learn: 1.1051354	total: 2m 52s	remai

...

665:	learn: 0.5021325	total: 28m 25s	remaining: 14m 15s
666:	learn: 0.5017473	total: 28m 27s	remaining: 14m 12s
667:	learn: 0.5013091	total: 28m 30s	remaining: 14m 10s
668:	learn: 0.5007921	total: 28m 32s	remaining: 14m 7s
669:	learn: 0.5004702	total: 28m 35s	remaining: 14m 4s
670:	learn: 0.4997404	total: 28m 37s	remaining: 14m 2s
671:	learn: 0.4990573	total: 28m 40s	remaining: 13m 59s
672:	learn: 0.4981552	total: 28m 43s	remaining: 13m 57s
673:	learn: 0.4974582	total: 28m 46s	remaining: 13m 54s
674:	learn: 0.4970074	total: 28m 49s	remaining: 13m 52s
675:	learn: 0.4966276	total: 28m 51s	remaining: 13m 50s
676:	learn: 0.4964282	total: 28m 54s	remaining: 13m 47s
677:	learn: 0.4961127	total: 28m 57s	remaining: 13m 44s
678:	learn: 0.4956497	total: 28m 59s	remaining: 13m 42s
679:	learn: 0.4951584	total: 29m 2s	remaining: 13m 39s
680:	learn: 0.4947378	total: 29m 4s	remaining: 13m 37s
681:	learn: 0.4942613	total: 29m 7s	remaining: 13m 34s
682:	learn: 0.4937741	total: 29m 10s	remaining: 13m 32

814:	learn: 0.4365092	total: 34m 53s	remaining: 7m 55s
815:	learn: 0.4362243	total: 34m 56s	remaining: 7m 52s
816:	learn: 0.4357833	total: 34m 59s	remaining: 7m 50s
817:	learn: 0.4353824	total: 35m 2s	remaining: 7m 47s
818:	learn: 0.4350935	total: 35m 4s	remaining: 7m 45s
819:	learn: 0.4346044	total: 35m 7s	remaining: 7m 42s
820:	learn: 0.4341899	total: 35m 10s	remaining: 7m 40s
821:	learn: 0.4335638	total: 35m 13s	remaining: 7m 37s
822:	learn: 0.4331974	total: 35m 16s	remaining: 7m 35s
823:	learn: 0.4329130	total: 35m 18s	remaining: 7m 32s
824:	learn: 0.4324507	total: 35m 21s	remaining: 7m 30s
825:	learn: 0.4321389	total: 35m 24s	remaining: 7m 27s
826:	learn: 0.4314913	total: 35m 27s	remaining: 7m 25s
827:	learn: 0.4307034	total: 35m 30s	remaining: 7m 22s
828:	learn: 0.4300451	total: 35m 33s	remaining: 7m 20s
829:	learn: 0.4295188	total: 35m 36s	remaining: 7m 17s
830:	learn: 0.4288185	total: 35m 38s	remaining: 7m 14s
831:	learn: 0.4284579	total: 35m 41s	remaining: 7m 12s
832:	learn: 0

965:	learn: 0.3783668	total: 41m 26s	remaining: 1m 27s
966:	learn: 0.3779572	total: 41m 28s	remaining: 1m 24s
967:	learn: 0.3777639	total: 41m 31s	remaining: 1m 22s
968:	learn: 0.3774444	total: 41m 33s	remaining: 1m 19s
969:	learn: 0.3772199	total: 41m 36s	remaining: 1m 17s
970:	learn: 0.3770094	total: 41m 38s	remaining: 1m 14s
971:	learn: 0.3767809	total: 41m 40s	remaining: 1m 12s
972:	learn: 0.3762611	total: 41m 43s	remaining: 1m 9s
973:	learn: 0.3759004	total: 41m 46s	remaining: 1m 6s
974:	learn: 0.3755158	total: 41m 48s	remaining: 1m 4s
975:	learn: 0.3750317	total: 41m 51s	remaining: 1m 1s
976:	learn: 0.3748120	total: 41m 53s	remaining: 59.2s
977:	learn: 0.3744985	total: 41m 55s	remaining: 56.6s
978:	learn: 0.3743757	total: 41m 58s	remaining: 54s
979:	learn: 0.3741143	total: 42m	remaining: 51.4s
980:	learn: 0.3738806	total: 42m 3s	remaining: 48.9s
981:	learn: 0.3737106	total: 42m 6s	remaining: 46.3s
982:	learn: 0.3730456	total: 42m 8s	remaining: 43.7s
983:	learn: 0.3728774	total: 4

,dim,CatBoost
chroma_cens,84.000000,43.37%
chroma_cqt,84.000000,45.36%
chroma_stft,84.000000,48.85%
mfcc,140.000000,60.86%
rmse,7.000000,40.15%
spectral_bandwidth,7.000000,44.15%
spectral_centroid,7.000000,46.95%
spectral_contrast,49.000000,54.49%
spectral_rolloff,7.000000,48.08%
tonnetz,42.000000,43.30%


,CatBoost
chroma_cens,3570.5156
chroma_cqt,3278.2969
chroma_stft,3213.6562
mfcc,6071.7500
rmse,343.0000
spectral_bandwidth,350.0000
spectral_centroid,344.0000
spectral_contrast,2167.0312
spectral_rolloff,333.0000
tonnetz,1837.8750


In [14]:
scores.to_csv('./catboost_scores.csv')
times.to_csv('./catboost_times.csv')

In [19]:
from catboost import CatBoostClassifier

classifiers = {
    'CatBoost': CatBoostClassifier()
}

feature_sets = {
   'echonest_audio': ('echonest', 'audio_features'),
   'echonest_social': ('echonest', 'social_features'),
   'echonest_temporal': ('echonest', 'temporal_features'),
   'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
   'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
# for name in features.columns.levels[0]:
#     feature_sets[name] = name
# feature_sets.update({
#     'mfcc/contrast': ['mfcc', 'spectral_contrast'],
#     'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
#     'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
#     'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
#     'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
#     'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
#     'all_non-echonest': list(features.columns.levels[0])
# })

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

c:\users\david\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
155297       -0.007668  0.104604 -0.511487  0.857308  0.266202 -0.355186   
155298       -0.956019 -0.961210 -0.447053 -0.689531 -0.633071 -0.430116   
155306        0.644815 -0.879404 -1.149232 -0.655986 -0.848881 -0.672240   
155307       -0.446937  1.235004 -0.250855 -0.612224 -1.197387 -1.118230   
155314       -0.121936 -0.348523 -0.555810 -0.607600 -0.823013 -0.369621   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
155297     -0.273640 -0.324727 -0.106368 -1.146157  ...  0.141844  0.024080   
155298     -0.031147 -0.758652 -1.093452  0.083694  ...  0.177286  0.028411   
155306      0.193149  0.069103 -0.179058 -0.713318  ...  0.125615  0.027110   
155307      0.716520 -0.878211 -0.637801  0.188355  ...  0.099894  0.019141   
155314     -0.573967  0.554874 -0.905412 -1.004594  ...  0.089492  0.019214   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
155297      0.031020  70.604774  0.850586  0.034100  0.020996  0.001953   
155298      0.025631  45.445824  0.303711  0.024058  0.019531  0.000000   
155306      0.028815  17.076239  0.247070  0.031542  0.025879  0.004883   
155307      0.033045   6.458911  0.344727  0.069199  0.063477  0.001953   
155314      0.018619   8.916731  0.227051  0.042894  0.039551  0.006348   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
155297      8.380918  0.088828  
155298      5.786194  0.023428  
155306      3.179079  0.022023  
155307      1.996085  0.043098  
155314      1.989080  0.019288  

[25000 rows x 518 columns]

KeyError: ('echonest', 'audio_features')

In [36]:
features_all = features.join(echonest, how='inner').sort_index(axis=1)
features_all.index
tracks = tracks[tracks.index.isin(features_all.index)]
features_all = features_all[features_all.index.isin(tracks.index)]

tracks.shape, features_all.shape

((5281, 52), (5281, 767))

In [37]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))

4167 training examples, 585 validation examples, 529 testing examples
Top genres (12): ['Blues', 'Classical', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Old-Time / Historic', 'Pop', 'Rock']
All genres (86): [2, 3, 4, 5, 6, 8, 10, 12, 15, 17, 18, 21, 25, 26, 31, 33, 36, 37, 38, 45, 46, 53, 58, 64, 66, 70, 71, 74, 79, 85, 86, 88, 89, 90, 92, 94, 97, 98, 100, 102, 109, 111, 113, 118, 130, 167, 171, 172, 177, 180, 181, 182, 184, 185, 187, 232, 236, 240, 286, 296, 297, 314, 322, 337, 359, 360, 361, 362, 400, 401, 404, 443, 468, 491, 495, 502, 504, 514, 524, 538, 539, 602, 619, 811, 906, 1235]


In [38]:
def pre_process(tracks, features, columns, multi_label=False, verbose=False):
    if not multi_label:
        # Assign an integer value to each genre.
        enc = LabelEncoder()
        labels = tracks['track', 'genre_top']
        #y = enc.fit_transform(tracks['track', 'genre_top'])
    else:
        # Create an indicator matrix.
        enc = MultiLabelBinarizer()
        labels = tracks['track', 'genres_all']
        #labels = tracks['track', 'genres']

    # Split in training, validation and testing sets.
    y_train = enc.fit_transform(labels[train])
    y_val = enc.transform(labels[val])
    y_test = enc.transform(labels[test])
    display(features)
    X_train = features.loc[train, columns].to_numpy()
    X_val = features.loc[val, columns].to_numpy()
    X_test = features.loc[test, columns].to_numpy()
    
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    
    # Standardize features by removing the mean and scaling to unit variance.
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    scaler.transform(X_val)
    scaler.transform(X_test)
    
    return y_train, y_val, y_test, X_train, X_val, X_test

def test_classifiers_features(classifiers, feature_sets, multi_label=False):
    columns = list(classifiers.keys()).insert(0, 'dim')
    scores = pd.DataFrame(columns=columns, index=feature_sets.keys())
    times = pd.DataFrame(columns=classifiers.keys(), index=feature_sets.keys())
    for fset_name, fset in tqdm_notebook(feature_sets.items(), desc='features'):
        y_train, y_val, y_test, X_train, X_val, X_test = pre_process(tracks, features_all, fset, multi_label)
        scores.loc[fset_name, 'dim'] = X_train.shape[1]
        for clf_name, clf in classifiers.items():  # tqdm_notebook(classifiers.items(), desc='classifiers', leave=False):
            t = time.process_time()
            clf.fit(X_train, y_train)
            score = clf.score(X_test, y_test)
            scores.loc[fset_name, clf_name] = score
            times.loc[fset_name, clf_name] = time.process_time() - t
    return scores, times

def format_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: yellow' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.2%}', subset=pd.IndexSlice[:, scores.columns[1]:])

from catboost import CatBoostClassifier

classifiers = {
    'CatBoost': CatBoostClassifier()
}

feature_sets = {
   'echonest_audio': ('echonest', 'audio_features'),
   'echonest_social': ('echonest', 'social_features'),
   'echonest_temporal': ('echonest', 'temporal_features'),
   'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
   'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
# for name in features.columns.levels[0]:
#     feature_sets[name] = name
# feature_sets.update({
#     'mfcc/contrast': ['mfcc', 'spectral_contrast'],
#     'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
#     'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
#     'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
#     'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
#     'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
#     'all_non-echonest': list(features.columns.levels[0])
# })

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

c:\users\david\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
124184        3.131895  1.432647  0.135905  0.956442  2.622694  1.607481   
124185        0.057650 -0.485441 -0.027666  0.229733 -0.519496 -0.707661   
124186       -0.336783 -0.324891 -0.099537 -0.268217 -0.285071 -0.548783   
124397       -0.879794 -0.399841 -0.899070  0.034932 -0.160796 -0.414672   
124402       -0.267039 -0.769515 -0.053341 -0.499768 -0.929167 -0.362893   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
124184      1.492101  1.941954  1.049861  3.875657  ...  0.065097  0.023337   
124185      0.415453  0.146410  0.686083  1.872930  ...  0.082299  0.025445   
124186     -0.681100  1.028564 -0.404963 -0.319033  ...  0.118877  0.019319   
124397      0.004725 -0.433029 -0.710138 -0.349848  ...  0.131294  0.031716   
124402     -0.758473  0.105333 -0.688703 -0.830325  ...  0.113713  0.027033   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
124184      0.017254   8.094973  0.340820  0.062146  0.054688  0.000000   
124185      0.018012  12.319770  0.306641  0.056615  0.052246  0.000000   
124186      0.032479   4.129772  0.238770  0.061523  0.062988  0.000000   
124397      0.024042   1.269326  0.165527  0.046637  0.045898  0.010254   
124402      0.030686  14.937521  0.242188  0.037816  0.034668  0.010742   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
124184      2.337578  0.035784  
124185      2.835066  0.034956  
124186      0.930179  0.029473  
124397      0.575714  0.013731  
124402      2.733083  0.018351  

[5281 rows x 767 columns]

Learning rate set to 0.085101
0:	learn: 2.2341120	total: 45.6ms	remaining: 45.6s
1:	learn: 2.0647971	total: 63.3ms	remaining: 31.6s
2:	learn: 1.9486157	total: 80.4ms	remaining: 26.7s
3:	learn: 1.8452812	total: 96.7ms	remaining: 24.1s
4:	learn: 1.7630641	total: 113ms	remaining: 22.5s
5:	learn: 1.6903822	total: 132ms	remaining: 21.8s
6:	learn: 1.6325025	total: 149ms	remaining: 21.1s
7:	learn: 1.5807240	total: 165ms	remaining: 20.5s
8:	learn: 1.5330475	total: 184ms	remaining: 20.2s
9:	learn: 1.4944822	total: 201ms	remaining: 19.9s
10:	learn: 1.4622315	total: 217ms	remaining: 19.5s
11:	learn: 1.4304473	total: 234ms	remaining: 19.2s
12:	learn: 1.4024876	total: 252ms	remaining: 19.1s
13:	learn: 1.3754745	total: 269ms	remaining: 18.9s
14:	learn: 1.3519131	total: 285ms	remaining: 18.7s
15:	learn: 1.3300531	total: 301ms	remaining: 18.5s
16:	learn: 1.3113952	total: 318ms	remaining: 18.4s
17:	learn: 1.2932024	total: 334ms	remaining: 18.2s
18:	learn: 1.2761995	total: 351ms	remaining: 18.1s
19:	lea

165:	learn: 0.8040394	total: 2.98s	remaining: 14.9s
166:	learn: 0.8024257	total: 2.99s	remaining: 14.9s
167:	learn: 0.8007042	total: 3.01s	remaining: 14.9s
168:	learn: 0.7990825	total: 3.02s	remaining: 14.9s
169:	learn: 0.7978290	total: 3.04s	remaining: 14.8s
170:	learn: 0.7963866	total: 3.06s	remaining: 14.8s
171:	learn: 0.7951015	total: 3.07s	remaining: 14.8s
172:	learn: 0.7925463	total: 3.09s	remaining: 14.8s
173:	learn: 0.7915539	total: 3.11s	remaining: 14.7s
174:	learn: 0.7891081	total: 3.12s	remaining: 14.7s
175:	learn: 0.7876880	total: 3.14s	remaining: 14.7s
176:	learn: 0.7868793	total: 3.16s	remaining: 14.7s
177:	learn: 0.7847562	total: 3.18s	remaining: 14.7s
178:	learn: 0.7837825	total: 3.19s	remaining: 14.6s
179:	learn: 0.7827390	total: 3.21s	remaining: 14.6s
180:	learn: 0.7815442	total: 3.23s	remaining: 14.6s
181:	learn: 0.7799192	total: 3.25s	remaining: 14.6s
182:	learn: 0.7791610	total: 3.26s	remaining: 14.6s
183:	learn: 0.7774839	total: 3.28s	remaining: 14.5s
184:	learn: 

325:	learn: 0.6161553	total: 5.75s	remaining: 11.9s
326:	learn: 0.6150965	total: 5.77s	remaining: 11.9s
327:	learn: 0.6143603	total: 5.79s	remaining: 11.9s
328:	learn: 0.6136706	total: 5.8s	remaining: 11.8s
329:	learn: 0.6118716	total: 5.82s	remaining: 11.8s
330:	learn: 0.6112610	total: 5.84s	remaining: 11.8s
331:	learn: 0.6102648	total: 5.85s	remaining: 11.8s
332:	learn: 0.6091501	total: 5.87s	remaining: 11.8s
333:	learn: 0.6078678	total: 5.89s	remaining: 11.7s
334:	learn: 0.6072177	total: 5.9s	remaining: 11.7s
335:	learn: 0.6067416	total: 5.92s	remaining: 11.7s
336:	learn: 0.6057667	total: 5.93s	remaining: 11.7s
337:	learn: 0.6045815	total: 5.95s	remaining: 11.7s
338:	learn: 0.6039641	total: 5.98s	remaining: 11.7s
339:	learn: 0.6032239	total: 6s	remaining: 11.6s
340:	learn: 0.6024849	total: 6.01s	remaining: 11.6s
341:	learn: 0.6017783	total: 6.03s	remaining: 11.6s
342:	learn: 0.6005180	total: 6.04s	remaining: 11.6s
343:	learn: 0.6000294	total: 6.06s	remaining: 11.6s
344:	learn: 0.599

...

618:	learn: 0.1735223	total: 4m 37s	remaining: 2m 50s
619:	learn: 0.1730011	total: 4m 38s	remaining: 2m 50s
620:	learn: 0.1727875	total: 4m 38s	remaining: 2m 50s
621:	learn: 0.1724704	total: 4m 39s	remaining: 2m 49s
622:	learn: 0.1721980	total: 4m 39s	remaining: 2m 49s
623:	learn: 0.1719120	total: 4m 39s	remaining: 2m 48s
624:	learn: 0.1716584	total: 4m 40s	remaining: 2m 48s
625:	learn: 0.1712815	total: 4m 40s	remaining: 2m 47s
626:	learn: 0.1709827	total: 4m 41s	remaining: 2m 47s
627:	learn: 0.1707796	total: 4m 41s	remaining: 2m 46s
628:	learn: 0.1704308	total: 4m 42s	remaining: 2m 46s
629:	learn: 0.1701789	total: 4m 43s	remaining: 2m 46s
630:	learn: 0.1700495	total: 4m 43s	remaining: 2m 45s
631:	learn: 0.1697600	total: 4m 44s	remaining: 2m 45s
632:	learn: 0.1695314	total: 4m 44s	remaining: 2m 44s
633:	learn: 0.1693415	total: 4m 45s	remaining: 2m 44s
634:	learn: 0.1691557	total: 4m 45s	remaining: 2m 44s
635:	learn: 0.1689390	total: 4m 45s	remaining: 2m 43s
636:	learn: 0.1684714	total:

771:	learn: 0.1340697	total: 5m 49s	remaining: 1m 43s
772:	learn: 0.1337967	total: 5m 49s	remaining: 1m 42s
773:	learn: 0.1335538	total: 5m 50s	remaining: 1m 42s
774:	learn: 0.1334188	total: 5m 50s	remaining: 1m 41s
775:	learn: 0.1332019	total: 5m 51s	remaining: 1m 41s
776:	learn: 0.1330263	total: 5m 51s	remaining: 1m 40s
777:	learn: 0.1328034	total: 5m 52s	remaining: 1m 40s
778:	learn: 0.1326045	total: 5m 52s	remaining: 1m 39s
779:	learn: 0.1323306	total: 5m 52s	remaining: 1m 39s
780:	learn: 0.1320587	total: 5m 53s	remaining: 1m 39s
781:	learn: 0.1319290	total: 5m 53s	remaining: 1m 38s
782:	learn: 0.1317839	total: 5m 54s	remaining: 1m 38s
783:	learn: 0.1314399	total: 5m 54s	remaining: 1m 37s
784:	learn: 0.1311322	total: 5m 55s	remaining: 1m 37s
785:	learn: 0.1308601	total: 5m 55s	remaining: 1m 36s
786:	learn: 0.1304825	total: 5m 56s	remaining: 1m 36s
787:	learn: 0.1304319	total: 5m 56s	remaining: 1m 35s
788:	learn: 0.1302077	total: 5m 56s	remaining: 1m 35s
789:	learn: 0.1298547	total:

926:	learn: 0.1046399	total: 7m 1s	remaining: 33.2s
927:	learn: 0.1045333	total: 7m 2s	remaining: 32.7s
928:	learn: 0.1044358	total: 7m 2s	remaining: 32.3s
929:	learn: 0.1043714	total: 7m 2s	remaining: 31.8s
930:	learn: 0.1042378	total: 7m 3s	remaining: 31.4s
931:	learn: 0.1040947	total: 7m 3s	remaining: 30.9s
932:	learn: 0.1040227	total: 7m 4s	remaining: 30.5s
933:	learn: 0.1039444	total: 7m 4s	remaining: 30s
934:	learn: 0.1037378	total: 7m 5s	remaining: 29.6s
935:	learn: 0.1036182	total: 7m 5s	remaining: 29.1s
936:	learn: 0.1033945	total: 7m 6s	remaining: 28.7s
937:	learn: 0.1032878	total: 7m 6s	remaining: 28.2s
938:	learn: 0.1031445	total: 7m 7s	remaining: 27.7s
939:	learn: 0.1029165	total: 7m 7s	remaining: 27.3s
940:	learn: 0.1026431	total: 7m 8s	remaining: 26.8s
941:	learn: 0.1024215	total: 7m 8s	remaining: 26.4s
942:	learn: 0.1022298	total: 7m 8s	remaining: 25.9s
943:	learn: 0.1021396	total: 7m 9s	remaining: 25.5s
944:	learn: 0.1017495	total: 7m 9s	remaining: 25s
945:	learn: 0.10

,dim,CatBoost
echonest_audio,8.000000,57.09%
echonest_social,5.000000,32.70%
echonest_temporal,224.000000,65.41%
echonest_audio/social,13.000000,57.84%
echonest_all,237.000000,70.32%


,CatBoost
echonest_audio,123.3750
echonest_social,82.8906
echonest_temporal,3911.0000
echonest_audio/social,258.8281
echonest_all,4043.4375


In [39]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

classifiers = {
    'GradientBoost': GradientBoostingClassifier(max_depth=5, n_estimators=10),
    'XGB': XGBClassifier(max_depth=5, n_estimators=10)
}

feature_sets = {
   'echonest_audio': ('echonest', 'audio_features'),
   'echonest_social': ('echonest', 'social_features'),
   'echonest_temporal': ('echonest', 'temporal_features'),
   'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
   'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
# for name in features.columns.levels[0]:
#     feature_sets[name] = name
# feature_sets.update({
#     'mfcc/contrast': ['mfcc', 'spectral_contrast'],
#     'mfcc/contrast/chroma': ['mfcc', 'spectral_contrast', 'chroma_cens'],
#     'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
#     'mfcc/contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
#     'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
#     'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
#     'all_non-echonest': list(features.columns.levels[0])
# })

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

c:\users\david\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
124184        3.131895  1.432647  0.135905  0.956442  2.622694  1.607481   
124185        0.057650 -0.485441 -0.027666  0.229733 -0.519496 -0.707661   
124186       -0.336783 -0.324891 -0.099537 -0.268217 -0.285071 -0.548783   
124397       -0.879794 -0.399841 -0.899070  0.034932 -0.160796 -0.414672   
124402       -0.267039 -0.769515 -0.053341 -0.499768 -0.929167 -0.362893   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
124184      1.492101  1.941954  1.049861  3.875657  ...  0.065097  0.023337   
124185      0.415453  0.146410  0.686083  1.872930  ...  0.082299  0.025445   
124186     -0.681100  1.028564 -0.404963 -0.319033  ...  0.118877  0.019319   
124397      0.004725 -0.433029 -0.710138 -0.349848  ...  0.131294  0.031716   
124402     -0.758473  0.105333 -0.688703 -0.830325  ...  0.113713  0.027033   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
124184      0.017254   8.094973  0.340820  0.062146  0.054688  0.000000   
124185      0.018012  12.319770  0.306641  0.056615  0.052246  0.000000   
124186      0.032479   4.129772  0.238770  0.061523  0.062988  0.000000   
124397      0.024042   1.269326  0.165527  0.046637  0.045898  0.010254   
124402      0.030686  14.937521  0.242188  0.037816  0.034668  0.010742   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
124184      2.337578  0.035784  
124185      2.835066  0.034956  
124186      0.930179  0.029473  
124397      0.575714  0.013731  
124402      2.733083  0.018351  

[5281 rows x 767 columns]

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
124184        3.131895  1.432647  0.135905  0.956442  2.622694  1.607481   
124185        0.057650 -0.485441 -0.027666  0.229733 -0.519496 -0.707661   
124186       -0.336783 -0.324891 -0.099537 -0.268217 -0.285071 -0.548783   
124397       -0.879794 -0.399841 -0.899070  0.034932 -0.160796 -0.414672   
124402       -0.267039 -0.769515 -0.053341 -0.499768 -0.929167 -0.362893   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
124184      1.492101  1.941954  1.049861  3.875657  ...  0.065097  0.023337   
124185      0.415453  0.146410  0.686083  1.872930  ...  0.082299  0.025445   
124186     -0.681100  1.028564 -0.404963 -0.319033  ...  0.118877  0.019319   
124397      0.004725 -0.433029 -0.710138 -0.349848  ...  0.131294  0.031716   
124402     -0.758473  0.105333 -0.688703 -0.830325  ...  0.113713  0.027033   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
124184      0.017254   8.094973  0.340820  0.062146  0.054688  0.000000   
124185      0.018012  12.319770  0.306641  0.056615  0.052246  0.000000   
124186      0.032479   4.129772  0.238770  0.061523  0.062988  0.000000   
124397      0.024042   1.269326  0.165527  0.046637  0.045898  0.010254   
124402      0.030686  14.937521  0.242188  0.037816  0.034668  0.010742   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
124184      2.337578  0.035784  
124185      2.835066  0.034956  
124186      0.930179  0.029473  
124397      0.575714  0.013731  
124402      2.733083  0.018351  

[5281 rows x 767 columns]

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
124184        3.131895  1.432647  0.135905  0.956442  2.622694  1.607481   
124185        0.057650 -0.485441 -0.027666  0.229733 -0.519496 -0.707661   
124186       -0.336783 -0.324891 -0.099537 -0.268217 -0.285071 -0.548783   
124397       -0.879794 -0.399841 -0.899070  0.034932 -0.160796 -0.414672   
124402       -0.267039 -0.769515 -0.053341 -0.499768 -0.929167 -0.362893   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
124184      1.492101  1.941954  1.049861  3.875657  ...  0.065097  0.023337   
124185      0.415453  0.146410  0.686083  1.872930  ...  0.082299  0.025445   
124186     -0.681100  1.028564 -0.404963 -0.319033  ...  0.118877  0.019319   
124397      0.004725 -0.433029 -0.710138 -0.349848  ...  0.131294  0.031716   
124402     -0.758473  0.105333 -0.688703 -0.830325  ...  0.113713  0.027033   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
124184      0.017254   8.094973  0.340820  0.062146  0.054688  0.000000   
124185      0.018012  12.319770  0.306641  0.056615  0.052246  0.000000   
124186      0.032479   4.129772  0.238770  0.061523  0.062988  0.000000   
124397      0.024042   1.269326  0.165527  0.046637  0.045898  0.010254   
124402      0.030686  14.937521  0.242188  0.037816  0.034668  0.010742   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
124184      2.337578  0.035784  
124185      2.835066  0.034956  
124186      0.930179  0.029473  
124397      0.575714  0.013731  
124402      2.733083  0.018351  

[5281 rows x 767 columns]

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
124184        3.131895  1.432647  0.135905  0.956442  2.622694  1.607481   
124185        0.057650 -0.485441 -0.027666  0.229733 -0.519496 -0.707661   
124186       -0.336783 -0.324891 -0.099537 -0.268217 -0.285071 -0.548783   
124397       -0.879794 -0.399841 -0.899070  0.034932 -0.160796 -0.414672   
124402       -0.267039 -0.769515 -0.053341 -0.499768 -0.929167 -0.362893   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
124184      1.492101  1.941954  1.049861  3.875657  ...  0.065097  0.023337   
124185      0.415453  0.146410  0.686083  1.872930  ...  0.082299  0.025445   
124186     -0.681100  1.028564 -0.404963 -0.319033  ...  0.118877  0.019319   
124397      0.004725 -0.433029 -0.710138 -0.349848  ...  0.131294  0.031716   
124402     -0.758473  0.105333 -0.688703 -0.830325  ...  0.113713  0.027033   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
124184      0.017254   8.094973  0.340820  0.062146  0.054688  0.000000   
124185      0.018012  12.319770  0.306641  0.056615  0.052246  0.000000   
124186      0.032479   4.129772  0.238770  0.061523  0.062988  0.000000   
124397      0.024042   1.269326  0.165527  0.046637  0.045898  0.010254   
124402      0.030686  14.937521  0.242188  0.037816  0.034668  0.010742   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
124184      2.337578  0.035784  
124185      2.835066  0.034956  
124186      0.930179  0.029473  
124397      0.575714  0.013731  
124402      2.733083  0.018351  

[5281 rows x 767 columns]

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   
...                ...       ...       ...       ...       ...       ...   
124184        3.131895  1.432647  0.135905  0.956442  2.622694  1.607481   
124185        0.057650 -0.485441 -0.027666  0.229733 -0.519496 -0.707661   
124186       -0.336783 -0.324891 -0.099537 -0.268217 -0.285071 -0.548783   
124397       -0.879794 -0.399841 -0.899070  0.034932 -0.160796 -0.414672   
124402       -0.267039 -0.769515 -0.053341 -0.499768 -0.929167 -0.362893   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   
...              ...       ...       ...       ...  ...       ...       ...   
124184      1.492101  1.941954  1.049861  3.875657  ...  0.065097  0.023337   
124185      0.415453  0.146410  0.686083  1.872930  ...  0.082299  0.025445   
124186     -0.681100  1.028564 -0.404963 -0.319033  ...  0.118877  0.019319   
124397      0.004725 -0.433029 -0.710138 -0.349848  ...  0.131294  0.031716   
124402     -0.758473  0.105333 -0.688703 -0.830325  ...  0.113713  0.027033   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.000000   
...              ...        ...       ...       ...       ...       ...   
124184      0.017254   8.094973  0.340820  0.062146  0.054688  0.000000   
124185      0.018012  12.319770  0.306641  0.056615  0.052246  0.000000   
124186      0.032479   4.129772  0.238770  0.061523  0.062988  0.000000   
124397      0.024042   1.269326  0.165527  0.046637  0.045898  0.010254   
124402      0.030686  14.937521  0.242188  0.037816  0.034668  0.010742   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
134         1.806106  0.054623  
...              ...       ...  
124184      2.337578  0.035784  
124185      2.835066  0.034956  
124186      0.930179  0.029473  
124397      0.575714  0.013731  
124402      2.733083  0.018351  

[5281 rows x 767 columns]

,dim,GradientBoost,XGB
echonest_audio,8.000000,54.44%,58.79%
echonest_social,5.000000,32.33%,28.36%
echonest_temporal,224.000000,55.39%,60.11%
echonest_audio/social,13.000000,58.41%,60.11%
echonest_all,237.000000,57.09%,62.19%


,GradientBoost,XGB
echonest_audio,2.4062,3.1250
echonest_social,1.1094,1.8594
echonest_temporal,51.0938,29.0312
echonest_audio/social,3.0781,3.0781
echonest_all,51.3281,28.0156
